<a href="https://colab.research.google.com/github/ihtanboga/VES-Deep-Learning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import matthews_corrcoef, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
#import shap

# Veri yükleme
df = pd.read_excel('df.xlsx')

# Train ve test verilerini ayırma
train_data = df[df['region'] == 1].copy()
test_data = df[df['region'] == 2].copy()

# Define categorical and numerical features first
categorical_features = ['Multifocal_PVC', 'Nonsustained_VT', 'gender', 'HTN', 'DM', 'Fullcompansasion']
numeric_features = ['pvc_percent', 'PVCQRS', 'EF', 'Age', 'PVC_Prematurity_index', 'QRS_ratio',
                    'mean_HR', 'symptom_duration', 'QTc_sinus', 'PVCCI_dispersion',
                    'CI_variability', 'PVC_Peak_QRS_duration', 'PVCCI', 'PVC_Compansatory_interval']

# Then perform label encoding
le = LabelEncoder()
for col in categorical_features:
    df[col] = le.fit_transform(df[col])


# Label Encoding için kategorik değişkenleri dönüştürme
#le = LabelEncoder()
#for col in categorical_features:
 #   train_data[col] = le.fit_transform(train_data[col])
 #   test_data[col] = le.transform(test_data[col])

# Standartlaştırma
scaler = StandardScaler()
train_data[numeric_features] = scaler.fit_transform(train_data[numeric_features])
test_data[numeric_features] = scaler.transform(test_data[numeric_features])

# X ve y ayırma
X_train = train_data[numeric_features + categorical_features]
y_train = train_data['Group']
X_test = test_data[numeric_features + categorical_features]
y_test = test_data['Group']

def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    """
    Model performans metriklerini hesaplar ve yazdırır
    """
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    print(f"\n{model_name} Performans Metrikleri:")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"AUC: {auc:.3f}")
    print(f"Sensitivity: {sensitivity:.3f}")
    print(f"Specificity: {specificity:.3f}")
    print(f"PPV: {ppv:.3f}")
    print(f"NPV: {npv:.3f}")
    print(f"F1 Score: {f1:.3f}")
    print(f"MCC: {mcc:.3f}")

    return auc

def plot_roc_curves(models_dict):
    """
    Tüm modeller için ROC eğrilerini çizer
    """
    plt.figure(figsize=(10, 8))
    for model_name, (model, y_pred_proba) in models_dict.items():
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        auc = roc_auc_score(y_test, y_pred_proba)
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.3f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for All Models')
    plt.legend()
    plt.grid(True)
    plt.show()

# Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    matthews_corrcoef, f1_score, roc_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# 1. Veri Yükleme ve Ön İşleme
# -----------------------------

# Veri yükleme
df = pd.read_excel('df.xlsx')

# Kategorik ve numerik değişkenleri ayırma
categorical_features = ['Multifocal_PVC', 'Nonsustained_VT', 'gender', 'HTN', 'DM', 'Fullcompansasion']
numeric_features = [
    'pvc_percent', 'PVCQRS', 'EF', 'Age', 'PVC_Prematurity_index',
    'QRS_ratio', 'mean_HR', 'symptom_duration', 'QTc_sinus',
    'PVCCI_dispersion', 'CI_variability', 'PVC_Peak_QRS_duration',
    'PVCCI', 'PVC_Compansatory_interval'
]

# Label Encoding için kategorik değişkenleri dönüştürme
le = LabelEncoder()
for col in categorical_features:
    df[col] = le.fit_transform(df[col])

# Train ve test verilerini ayırma
train_data = df[df['region'] == 1].copy()
test_data = df[df['region'] == 2].copy()

# Standartlaştırma
scaler = StandardScaler()
train_data[numeric_features] = scaler.fit_transform(train_data[numeric_features])
test_data[numeric_features] = scaler.transform(test_data[numeric_features])

# X ve y ayırma
X_train = train_data[numeric_features + categorical_features]
y_train = train_data['Group']
X_test = test_data[numeric_features + categorical_features]
y_test = test_data['Group']

# -----------------------------
# 2. Model Performans Fonksiyonu
# -----------------------------

def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    """ Model performans metriklerini hesaplar ve yazdırır """
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    print(f"\n{model_name} Performans Metrikleri:")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"AUC: {auc:.3f}")
    print(f"Sensitivity: {sensitivity:.3f}")
    print(f"Specificity: {specificity:.3f}")
    print(f"PPV: {ppv:.3f}")
    print(f"NPV: {npv:.3f}")
    print(f"F1 Score: {f1:.3f}")
    print(f"MCC: {mcc:.3f}")
    return auc

# -----------------------------
# 3. Hyperparameter Tuning ve Model Eğitimi
# -----------------------------

# Hiperparametre grid'i
param_grid = [
    {
        'penalty': ['l1'],
        'C': stats.loguniform(1e-5, 100),
        'solver': ['saga'],
        'class_weight': ['balanced', None],
        'max_iter': [1000],
        'random_state': [42]
    },
    {
        'penalty': ['elasticnet'],
        'C': stats.loguniform(1e-5, 100),
        'solver': ['saga'],
        'l1_ratio': stats.uniform(0, 1),
        'class_weight': ['balanced', None],
        'max_iter': [1000],
        'random_state': [42]
    },
    {
        'penalty': ['l2'],
        'C': stats.loguniform(1e-5, 100),
        'solver': ['saga'],
        'class_weight': ['balanced', None],
        'max_iter': [1000],
        'random_state': [42]
    }
]

# RandomizedSearchCV ile en iyi parametreleri bulma
lr = LogisticRegression()
lr_random = RandomizedSearchCV(
    estimator=lr,
    param_distributions=param_grid,
    n_iter=100,
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
lr_random.fit(X_train, y_train)

# En iyi model ile tahmin
best_lr = lr_random.best_estimator_
y_pred_lr = best_lr.predict(X_test)
y_pred_proba_lr = best_lr.predict_proba(X_test)[:, 1]

# Model performansını değerlendirme
auc_lr = evaluate_model(y_test, y_pred_lr, y_pred_proba_lr, "Logistic Regression")

# En iyi parametreleri yazdırma
print("\nEn İyi Logistic Regression Parametreleri:")
print(lr_random.best_params_)

# Sonuçları kaydetme
results_lr = pd.DataFrame({
    'True_Labels': y_test,
    'Predicted_Probabilities': y_pred_proba_lr
})
results_lr.to_csv('logistic_regression_results.csv', index=False)

# -----------------------------
# 4. SHAP Analizi
# -----------------------------

# SHAP Analizi
# LinearExplainer Logistic Regression için uygundur
explainer = shap.LinearExplainer(best_lr, X_train)

# SHAP değerlerini hesapla
shap_values = explainer.shap_values(X_test)

# SHAP Summary Plot
shap.summary_plot(shap_values, X_test, feature_names=X_test.columns)

# SHAP Feature Importance (Bar Plot)
shap.summary_plot(shap_values, X_test, feature_names=X_test.columns, plot_type="bar")



In [ ]:
import joblib

# Model, Scaler ve Encoder'ı kaydetme
joblib.dump(best_lr, 'logistic_regression_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(le, 'label_encoder.pkl')

print("Model, scaler ve encoder başarıyla dosyalara kaydedildi.")


Model, scaler ve encoder başarıyla dosyalara kaydedildi.


# Multi-layer Perceptron

In [ ]:
# -----------------------------
# 1. Gerekli Kütüphanelerin İçe Aktarılması
# -----------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    matthews_corrcoef, f1_score, roc_curve, classification_report
)
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn.calibration import calibration_curve
from scipy.stats import loguniform
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# 2. Veri Yükleme ve Ön İşleme
# -----------------------------

# Veri yükleme
df = pd.read_excel('df.xlsx')

# Kategorik ve numerik değişkenleri ayırma
categorical_features = ['Multifocal_PVC', 'Nonsustained_VT', 'gender', 'HTN', 'DM', 'Fullcompansasion']
numeric_features = [
    'pvc_percent', 'PVCQRS', 'EF', 'Age', 'PVC_Prematurity_index',
    'QRS_ratio', 'mean_HR', 'symptom_duration', 'QTc_sinus',
    'PVCCI_dispersion', 'CI_variability', 'PVC_Peak_QRS_duration',
    'PVCCI', 'PVC_Compansatory_interval'
]

# Label Encoding için kategorik değişkenleri dönüştürme
le = LabelEncoder()
for col in categorical_features:
    df[col] = le.fit_transform(df[col])

# Hedef değişkenin sınıflarını kontrol etme
print("Train setindeki benzersiz sınıflar:", df[df['region'] == 1]['Group'].unique())
print("Test setindeki benzersiz sınıflar:", df[df['region'] == 2]['Group'].unique())

# Train ve test verilerini ayırma
train_data = df[df['region'] == 1].copy()
test_data = df[df['region'] == 2].copy()

# Standartlaştırma
scaler = StandardScaler()
train_data[numeric_features] = scaler.fit_transform(train_data[numeric_features])
test_data[numeric_features] = scaler.transform(test_data[numeric_features])

# X ve y ayırma
X_train = train_data[numeric_features + categorical_features]
y_train = train_data['Group']
X_test = test_data[numeric_features + categorical_features]
y_test = test_data['Group']

# -----------------------------
# 3. Model Performansını Değerlendirmek İçin Fonksiyon
# -----------------------------

def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    """
    Model performansını değerlendirmek için çeşitli metrikler hesaplar ve yazdırır.
    """
    auc = roc_auc_score(y_true, y_pred_proba)
    accuracy = accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"{model_name} Performans Metrikleri:")
    print(f"AUC: {auc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Matthews Corr Coef: {mcc:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

    return auc

# -----------------------------
# 4. Hiperparametre Grid'i
# -----------------------------

param_grid = {
    'hidden_layer_sizes': [
        (50,), (100,), (50, 50), (100, 50), (100, 100),
        (50, 25), (100, 50, 25), (100, 100, 50)
    ],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': loguniform(1e-5, 1),
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': loguniform(1e-4, 1e-2),
    'max_iter': [1000],
    'early_stopping': [True],
    'validation_fraction': [0.1],
    'n_iter_no_change': [10],
    'random_state': [42]
}

# -----------------------------
# 5. RandomizedSearchCV ile En İyi Parametreleri Bulma
# -----------------------------

print("MLP modeli eğitiliyor...")
mlp = MLPClassifier()

mlp_random = RandomizedSearchCV(
    estimator=mlp,
    param_distributions=param_grid,
    n_iter=50,
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

mlp_random.fit(X_train, y_train)

# En iyi model ile tahmin
best_mlp = mlp_random.best_estimator_
y_pred_mlp = best_mlp.predict(X_test)
# İkili sınıflandırma mı çok sınıflı mı kontrol et
if best_mlp.predict_proba(X_test).shape[1] > 1:
    # İkili sınıflandırma için, genellikle ikinci sütun sınıf 1 için
    y_pred_proba_mlp = best_mlp.predict_proba(X_test)[:, 1]
else:
    # Tek sınıflı olasılık varsa
    y_pred_proba_mlp = best_mlp.predict_proba(X_test)[:, 0]

# -----------------------------
# 6. Model Performansını Değerlendirme
# -----------------------------

auc_mlp = evaluate_model(y_test, y_pred_mlp, y_pred_proba_mlp, "Multi-Layer Perceptron")

# -----------------------------
# 7. Feature Importance Hesaplama (Permutation Importance Kullanarak)
# -----------------------------

perm_importance = permutation_importance(
    best_mlp, X_test, y_test,
    n_repeats=10, random_state=42, n_jobs=-1
)

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': perm_importance.importances_mean
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Feature Importance Grafiği
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('MLP - Feature Importance (Permutation Importance)')
plt.tight_layout()
plt.show()

# -----------------------------
# 8. Learning Curves
# -----------------------------

plt.figure(figsize=(12, 4))

# Loss Curve
if hasattr(best_mlp, 'loss_curve_'):
    plt.subplot(1, 2, 1)
    plt.plot(best_mlp.loss_curve_)
    plt.title('Loss Curve')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.grid(True)

# Validation Score Curve
if hasattr(best_mlp, 'validation_scores_'):
    plt.subplot(1, 2, 2)
    plt.plot(best_mlp.validation_scores_)
    plt.title('Validation Score Curve')
    plt.xlabel('Iteration')
    plt.ylabel('Score')
    plt.grid(True)

plt.tight_layout()
plt.show()

# -----------------------------
# 9. Neural Network Architecture Visualization Fonksiyonu
# -----------------------------

def plot_neural_network(mlp):
    """
    MLPClassifier modelinin sinir ağı mimarisini görselleştirir.
    """
    n_layers = len(mlp.coefs_) + 1
    layer_sizes = [mlp.coefs_[0].shape[0]] + [layer.shape[1] for layer in mlp.coefs_]

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111)

    # Plot nodes
    for i, size in enumerate(layer_sizes):
        x = np.full(size, i)
        y = np.linspace(0, size-1, size)
        ax.scatter(x, y, s=100, zorder=4, color='skyblue', edgecolors='k')

        if i < len(layer_sizes)-1:
            # Plot connections
            for j in range(size):
                for k in range(layer_sizes[i+1]):
                    weight = mlp.coefs_[i][j, k]
                    color = 'red' if weight < 0 else 'blue'
                    alpha = min(abs(weight)*10, 1)  # Normalize alpha for better visibility
                    ax.plot([i, i+1], [j, k], color=color, alpha=alpha, linewidth=0.5)

    ax.set_xticks(range(n_layers))
    ax.set_xticklabels(['Input'] + [f'Hidden {i}' for i in range(1, n_layers-1)] + ['Output'])
    ax.set_yticks([])
    plt.title('Neural Network Architecture')
    plt.grid(False)
    plt.tight_layout()
    plt.show()

plot_neural_network(best_mlp)

# -----------------------------
# 10. Calibration Curve
# -----------------------------

prob_true, prob_pred = calibration_curve(y_test, y_pred_proba_mlp, n_bins=10)

plt.figure(figsize=(8, 6))
plt.plot(prob_pred, prob_true, marker='o', label='MLP')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect Calibration')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('True Probability')
plt.title('Calibration Curve - MLP')
plt.legend()
plt.grid(True)
plt.show()

# -----------------------------
# 11. Confusion Matrix ile Normalized Değerler
# -----------------------------

cm = confusion_matrix(y_test, y_pred_mlp)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, cmap='Blues', fmt='.2f',
            xticklabels=['Not Group', 'Group'],
            yticklabels=['Not Group', 'Group'])
plt.title('Normalized Confusion Matrix - MLP')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# -----------------------------
# 12. Decision Boundary Visualization Fonksiyonu
# -----------------------------

def plot_decision_boundary(model, X, y, feature1_idx=0, feature2_idx=1):
    """
    İki özellik kullanarak modelin karar sınırını görselleştirir.
    """
    # Sadece iki özelliği kullanarak karar sınırını görselleştiriyoruz
    h = .02  # Mesh grid adım boyutu
    x_min, x_max = X.iloc[:, feature1_idx].min() - 1, X.iloc[:, feature1_idx].max() + 1
    y_min, y_max = X.iloc[:, feature2_idx].min() - 1, X.iloc[:, feature2_idx].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Karar sınırını tahmin etmek için tüm grid noktalarını kullanıyoruz
    X_plot = pd.DataFrame(np.zeros((xx.ravel().shape[0], X.shape[1])), columns=X.columns)
    X_plot.iloc[:, feature1_idx] = xx.ravel()
    X_plot.iloc[:, feature2_idx] = yy.ravel()

    # Diğer özellikleri ortalama değerlerle dolduruyoruz
    for i in range(X.shape[1]):
        if i != feature1_idx and i != feature2_idx:
            X_plot.iloc[:, i] = X.iloc[:, i].mean()

    # Tahminler
    Z = model.predict(X_plot)
    Z = Z.reshape(xx.shape)

    # Plot
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='coolwarm')
    scatter = plt.scatter(X.iloc[:, feature1_idx], X.iloc[:, feature2_idx], c=y, cmap='coolwarm', edgecolor='k', alpha=0.7)
    plt.xlabel(X.columns[feature1_idx])
    plt.ylabel(X.columns[feature2_idx])
    plt.title('Decision Boundary - Top 2 Features')
    plt.legend(*scatter.legend_elements(), title="Classes")
    plt.show()

# -----------------------------
# 13. Karar Sınırı Görselleştirme
# -----------------------------

# Top 2 Özelliği Kullanarak Decision Boundary Çizimi
top_features = feature_importance.head(2)['feature'].values
feature1_idx = X_train.columns.get_loc(top_features[0])
feature2_idx = X_train.columns.get_loc(top_features[1])
plot_decision_boundary(best_mlp, X_test, y_test.values, feature1_idx, feature2_idx)

# -----------------------------
# 14. En İyi Parametreleri Yazdırma
# -----------------------------

print("\nEn İyi MLP Parametreleri:")
print(mlp_random.best_params_)

# -----------------------------
# 15. Model Complexity Analizi Fonksiyonu
# -----------------------------

def analyze_model_complexity(model, X_train, y_train, X_test, y_test):
    """
    Modelin karmaşıklığını ve eğitim ile test performansını analiz eder.
    """
    n_layers = len(model.hidden_layer_sizes)
    total_neurons = sum(model.hidden_layer_sizes)
    n_parameters = sum(coef.size for coef in model.coefs_)

    print("\nModel Complexity Analysis:")
    print(f"Number of hidden layers: {n_layers}")
    print(f"Total number of neurons in hidden layers: {total_neurons}")
    print(f"Total number of parameters: {n_parameters}")

    # Training vs test performance
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    print(f"\nTraining accuracy: {train_score:.3f}")
    print(f"Test accuracy: {test_score:.3f}")
    print(f"Overfitting gap: {train_score - test_score:.3f}")

analyze_model_complexity(best_mlp, X_train, y_train, X_test, y_test)

# -----------------------------
# 16. Sonuçları Kaydetme
# -----------------------------

results_mlp = pd.DataFrame({
    'True_Labels': y_test,
    'Predicted_Probabilities': y_pred_proba_mlp
})
results_mlp.to_csv('mlp_results.csv', index=False)
print("\nSonuçlar 'mlp_results.csv' dosyasına kaydedildi.")

# -----------------------------
# 17. Prediction Confidence Analysis
# -----------------------------

prediction_confidence = np.max(best_mlp.predict_proba(X_test), axis=1)
plt.figure(figsize=(8, 6))
sns.histplot(data=prediction_confidence, bins=30, kde=True)
plt.title('Distribution of Prediction Confidence')
plt.xlabel('Confidence')
plt.ylabel('Count')
plt.show()

# -----------------------------
# 18. SHAP Entegrasyonu
# -----------------------------

# SHAP değerlerini hesaplama
print("\nSHAP değerleri hesaplanıyor...")

# Hedef değişkenin sınıf sayısını kontrol etme
n_classes = len(np.unique(y_train))
print(f"Hedef değişkenin sınıf sayısı: {n_classes}")

# SHAP hesaplama
if n_classes == 2:
    # İkili sınıflandırma için
    explainer = shap.KernelExplainer(best_mlp.predict_proba, shap.sample(X_train, 100), link="logit")
    shap_values = explainer.shap_values(X_test, nsamples=100)

    # SHAP Summary Plot (Bar)
    print("SHAP Summary Plot oluşturuluyor...")
    if isinstance(shap_values, list):
        # SHAP değerleri bir liste ise, ikinci sınıf için kullan
        shap.summary_plot(shap_values[1], X_test, plot_type="bar")  # Sınıf 1 için özet plot
    else:
        # SHAP değerleri tek bir dizi ise, üçüncü boyutta sınıfı seç
        shap.summary_plot(shap_values[:, :, 1], X_test, plot_type="bar")  # Sınıf 1 için özet plot

    # SHAP Summary Plot (Scatter)
    print("SHAP Scatter Summary Plot oluşturuluyor...")
    if isinstance(shap_values, list):
        shap.summary_plot(shap_values[1], X_test)
    else:
        shap.summary_plot(shap_values[:, :, 1], X_test)

    # SHAP Dependence Plot
    top_feature = feature_importance.head(1)['feature'].values[0]
    print(f"{top_feature} için SHAP Dependence Plot oluşturuluyor...")
    if isinstance(shap_values, list):
        shap.dependence_plot(top_feature, shap_values[1], X_test)
    else:
        shap.dependence_plot(top_feature, shap_values[:, :, 1], X_test)

    # SHAP Force Plot (İlk birkaç örnek için)
    print("SHAP Force Plot oluşturuluyor...")
    shap.initjs()
    for i in range(min(3, X_test.shape[0])):  # İlk 3 örnek için
        if isinstance(shap_values, list):
            shap.force_plot(explainer.expected_value[1], shap_values[1][i], X_test.iloc[i], matplotlib=True)
        else:
            shap.force_plot(explainer.expected_value[1], shap_values[i, :, 1], X_test.iloc[i], matplotlib=True)
        plt.show()
else:
    # Çok sınıflı sınıflandırma için
    explainer = shap.KernelExplainer(best_mlp.predict_proba, shap.sample(X_train, 100), link="logit")
    shap_values = explainer.shap_values(X_test, nsamples=100)

    for class_idx in range(n_classes):
        print(f"\nSHAP Summary Plot oluşturuluyor - Sınıf {class_idx}:")
        if isinstance(shap_values, list):
            shap.summary_plot(shap_values[class_idx], X_test, plot_type="bar")
        else:
            shap.summary_plot(shap_values[:, :, class_idx], X_test, plot_type="bar")
        plt.title(f"SHAP Summary Plot - Class {class_idx}")
        plt.show()

        print(f"SHAP Scatter Summary Plot oluşturuluyor - Sınıf {class_idx}:")
        if isinstance(shap_values, list):
            shap.summary_plot(shap_values[class_idx], X_test)
        else:
            shap.summary_plot(shap_values[:, :, class_idx], X_test)
        plt.title(f"SHAP Scatter Summary Plot - Class {class_idx}")
        plt.show()

        # SHAP Dependence Plot
        top_feature = feature_importance.head(1)['feature'].values[0]
        print(f"{top_feature} için SHAP Dependence Plot oluşturuluyor - Sınıf {class_idx}:")
        if isinstance(shap_values, list):
            shap.dependence_plot(top_feature, shap_values[class_idx], X_test)
        else:
            shap.dependence_plot(top_feature, shap_values[:, :, class_idx], X_test)
        plt.title(f"SHAP Dependence Plot - Class {class_idx}")
        plt.show()

        # SHAP Force Plot (İlk birkaç örnek için)
        print(f"SHAP Force Plot oluşturuluyor - Sınıf {class_idx}:")
        shap.initjs()
        for i in range(min(3, X_test.shape[0])):  # İlk 3 örnek için
            if isinstance(shap_values, list):
                shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx][i], X_test.iloc[i], matplotlib=True)
            else:
                shap.force_plot(explainer.expected_value[class_idx], shap_values[i, :, class_idx], X_test.iloc[i], matplotlib=True)
            plt.show()


In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Scaler'ı kaydet
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [ ]:
import joblib

# Modeli bir dosyaya kaydet
joblib.dump(best_mlp, 'mlp_model.pkl')
print("Model başarıyla mlp_model.pkl dosyasına kaydedildi.")

Model başarıyla mlp_model.pkl dosyasına kaydedildi.


# TabTransformer

In [ ]:
!pip install pytorch-lightning

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report, matthews_corrcoef
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Tabular dataset class
class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.FloatTensor(X)
        if y is not None:
            self.y = torch.LongTensor(y)
        else:
            self.y = None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

# TabTransformer model class
class TabTransformer(pl.LightningModule):
    def __init__(self, input_dim, num_classes=2, d_model=64, nhead=4, num_layers=3, dropout=0.1):
        super().__init__()
        self.save_hyperparameters()  # Save the hyperparameters

        self.embedding = nn.Linear(input_dim, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            activation='gelu'
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, num_classes)
        )

        self.criterion = nn.CrossEntropyLoss()
        self.validation_outputs = []  # Save outputs for validation

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(0)  # Add sequence length dimension
        x = self.transformer_encoder(x)
        x = x.squeeze(0)  # Remove sequence length dimension
        return self.fc(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss)
        self.validation_outputs.append({'val_loss': loss, 'y_true': y, 'y_pred': torch.softmax(y_hat, dim=1)[:, 1]})

    def on_validation_epoch_end(self):
        avg_loss = torch.stack([x['val_loss'] for x in self.validation_outputs]).mean()
        y_true = torch.cat([x['y_true'] for x in self.validation_outputs])
        y_pred = torch.cat([x['y_pred'] for x in self.validation_outputs])
        auc = roc_auc_score(y_true.cpu(), y_pred.cpu())
        self.log('val_auc', auc)
        self.validation_outputs.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

def predict_proba(model, X):
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X.values)
        logits = model(X_tensor)
        probabilities = F.softmax(logits, dim=1)
    return probabilities.cpu().numpy()

# Model evaluation and metric calculation
def evaluate_model(y_true, y_pred, y_pred_proba):
    # Confusion matrix components
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)  # Recall
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0  # Positive Predictive Value
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Negative Predictive Value
    mcc = matthews_corrcoef(y_true, y_pred)

    # Print metrics
    print("\nModel Performance Summary:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.3f}")
    print(f"AUC: {roc_auc_score(y_true, y_pred_proba):.3f}")
    print(f"F1 Score: {f1_score(y_true, y_pred):.3f}")
    print(f"Sensitivity (Recall): {sensitivity:.3f}")
    print(f"Specificity: {specificity:.3f}")
    print(f"PPV (Precision): {ppv:.3f}")
    print(f"NPV: {npv:.3f}")
    print(f"MCC: {mcc:.3f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

# Data preparation
train_dataset = TabularDataset(X_train.values, y_train.values)
test_dataset = TabularDataset(X_test.values, y_test.values)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)

# Model training
model = TabTransformer(
    input_dim=X_train.shape[1],
    d_model=64,
    nhead=4,
    num_layers=3,
    dropout=0.1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min'
)

trainer = pl.Trainer(
    max_epochs=100,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    callbacks=[early_stopping]
)

trainer.fit(model, train_loader, test_loader)

# Model evaluation
model.eval()
y_pred_proba = []
with torch.no_grad():
    for batch in test_loader:
        x, _ = batch
        outputs = model(x)
        probas = torch.softmax(outputs, dim=1)[:, 1]
        y_pred_proba.extend(probas.cpu().numpy())

y_pred_proba = np.array(y_pred_proba)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate and display all metrics
evaluate_model(y_test, y_pred, y_pred_proba)

# Display best hyperparameters
print("\nBest Hyperparameters:")
print(model.hparams)

# Feature importance calculation and visualization
def calculate_permutation_importance(model, X, y, n_repeats=10):
    baseline_score = roc_auc_score(y, predict_proba(model, X)[:, 1])
    importances = []

    for column in X.columns:
        scores = []
        for _ in range(n_repeats):
            X_permuted = X.copy()
            X_permuted[column] = np.random.permutation(X_permuted[column])
            score = roc_auc_score(y, predict_proba(model, X_permuted)[:, 1])
            scores.append(baseline_score - score)
        importances.append(np.mean(scores))

    return importances

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': calculate_permutation_importance(model, X_test, y_test)
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('TabTransformer - Feature Importance (Permutation Importance)')
plt.tight_layout()
plt.show()

# Saving results
results_tab = pd.DataFrame({
    'True_Labels': y_test,
    'Predicted_Probabilities': y_pred_proba
})
results_tab.to_csv('tabtransformer_results.csv', index=False)


In [ ]:
import torch
import pickle

# Modelinizi kaydetmek
torch.save(model.state_dict(), 'tabtransformer_model.pth')
print("Model başarıyla tabtransformer_model.pth dosyasına kaydedildi.")

# Ölçekleyiciyi kaydetmek
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)
print("Scaler başarıyla scaler.pkl dosyasına kaydedildi.")

Model başarıyla tabtransformer_model.pth dosyasına kaydedildi.
Scaler başarıyla scaler.pkl dosyasına kaydedildi.


In [ ]:
import shap
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Set the backend to Agg for better file saving

def predict_proba_for_shap(X):
    model.eval()
    with torch.no_grad():
        if isinstance(X, pd.DataFrame):
            X_tensor = torch.FloatTensor(X.values)
        elif isinstance(X, np.ndarray):
            X_tensor = torch.FloatTensor(X)
        else:
            raise TypeError("Input must be either pandas DataFrame or numpy array")

        logits = model(X_tensor)
        probabilities = torch.softmax(logits, dim=1)
        return probabilities[:, 1].cpu().numpy()

# Create background dataset
n_background = 50
background = X_train.sample(n=n_background, random_state=42)

print("Creating SHAP explainer...")
explainer = shap.KernelExplainer(
    predict_proba_for_shap,
    background,
    link="logit",
    n_jobs=-1
)

# Sample test data
n_explain = 50
X_sample = X_test.sample(n=n_explain, random_state=42)

print("Calculating SHAP values...")
shap_values = explainer.shap_values(
    X_sample,
    nsamples=100,
    n_jobs=-1
)

print("Generating SHAP plots...")
def save_shap_plots():
    try:
        # Summary Dot Plot
        plt.clf()  # Clear the current figure
        fig_dot = plt.figure(figsize=(12, 8))
        shap.summary_plot(
            shap_values,
            X_sample,
            plot_type="dot",
            show=False,
            max_display=20
        )
        plt.title('SHAP Summary Plot (Dot)')
        plt.tight_layout()
        fig_dot.savefig('shap_summary_dot.png', bbox_inches='tight', dpi=300)
        plt.close(fig_dot)

        # Summary Bar Plot
        plt.clf()
        fig_bar = plt.figure(figsize=(12, 8))
        shap.summary_plot(
            shap_values,
            X_sample,
            plot_type="bar",
            show=False,
            max_display=20
        )
        plt.title('SHAP Summary Plot (Bar)')
        plt.tight_layout()
        fig_bar.savefig('shap_summary_bar.png', bbox_inches='tight', dpi=300)
        plt.close(fig_bar)

        # Feature importance calculation
        feature_importance = np.abs(shap_values).mean(0)
        most_important_idx = np.argmax(feature_importance)
        important_feature = X_test.columns[most_important_idx]

        # Dependence Plot
        plt.clf()
        fig_dep = plt.figure(figsize=(12, 8))
        shap.dependence_plot(
            most_important_idx,
            shap_values,
            X_sample,
            show=False
        )
        plt.title(f'SHAP Dependence Plot for {important_feature}')
        plt.tight_layout()
        fig_dep.savefig(f'shap_dependence_{important_feature}.png', bbox_inches='tight', dpi=300)
        plt.close(fig_dep)

        # Force Plot
        plt.clf()
        fig_force = plt.figure(figsize=(15, 3))
        shap.force_plot(
            explainer.expected_value,
            shap_values[0],
            X_sample.iloc[0],
            show=False,
            matplotlib=True
        )
        plt.title('SHAP Force Plot (Single Example)')
        plt.tight_layout()
        fig_force.savefig('shap_force_plot.png', bbox_inches='tight', dpi=300)
        plt.close(fig_force)

        # Save feature importance DataFrame
        shap_importance = pd.DataFrame({
            'feature': X_test.columns,
            'importance': feature_importance
        })
        shap_importance = shap_importance.sort_values('importance', ascending=False)
        shap_importance.to_csv('shap_importance.csv', index=False)

        print("\nFeature Importance based on SHAP values:")
        print(shap_importance)

        # Create a bar plot of feature importance
        plt.clf()
        fig_imp = plt.figure(figsize=(12, 8))
        plt.bar(range(len(feature_importance)), shap_importance['importance'])
        plt.xticks(range(len(feature_importance)), shap_importance['feature'], rotation=45, ha='right')
        plt.title('Feature Importance from SHAP Values')
        plt.xlabel('Features')
        plt.ylabel('Mean |SHAP Value|')
        plt.tight_layout()
        fig_imp.savefig('feature_importance_bar.png', bbox_inches='tight', dpi=300)
        plt.close(fig_imp)

    except Exception as e:
        print(f"Error generating plots: {str(e)}")
        raise

# Execute SHAP analysis
try:
    save_shap_plots()
    print("\nSHAP analysis completed successfully. The following files have been saved:")
    print("- shap_summary_dot.png")
    print("- shap_summary_bar.png")
    print(f"- shap_dependence_{X_test.columns[np.argmax(np.abs(shap_values).mean(0))]}.png")
    print("- shap_force_plot.png")
    print("- feature_importance_bar.png")
    print("- shap_importance.csv")
    print("- shap_values.csv")
except Exception as e:
    print(f"SHAP analysis failed: {str(e)}")

# Save SHAP values
shap_results = pd.DataFrame(
    shap_values,
    columns=X_test.columns
)
shap_results.to_csv('shap_values.csv', index=False)

# TabNet

In [ ]:
pip install pytorch-tabnet


In [ ]:
!pip install optuna

In [ ]:
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.calibration import calibration_curve
import optuna

# Assuming the data is already loaded and preprocessed as X and y
# Split data into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a function to evaluate the model
def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    auc = roc_auc_score(y_true, y_pred_proba)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"{model_name} - AUC: {auc:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
    return auc, accuracy, f1

# Optuna objective function
def objective(trial):
    params = {
        "n_d": trial.suggest_int("n_d", 8, 64, step=8),
        "n_a": trial.suggest_int("n_a", 8, 64, step=8),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True),
        "optimizer_fn": torch.optim.Adam,
        "optimizer_params": dict(lr=trial.suggest_float("lr", 1e-4, 1e-2, log=True)),
        "mask_type": trial.suggest_categorical("mask_type", ["entmax", "sparsemax"]),
        "n_shared": trial.suggest_int("n_shared", 1, 3),
        "n_independent": trial.suggest_int("n_independent", 1, 3),
        "scheduler_params": dict(
            mode="min",
            patience=trial.suggest_int("patienceScheduler", 3, 10),
            min_lr=1e-5,
            factor=0.5,
        ),
        "scheduler_fn": torch.optim.lr_scheduler.ReduceLROnPlateau,
        "verbose": 0
    }

    model = TabNetClassifier(**params)
    model.fit(
        X_train=X_train.values,
        y_train=y_train.values,
        eval_set=[(X_test.values, y_test.values)],
        eval_name=["valid"],
        patience=trial.suggest_int("patience", 10, 30),
        max_epochs=trial.suggest_int("epochs", 10, 100),
        batch_size=128,
        virtual_batch_size=64,
    )
    y_pred_proba = model.predict_proba(X_test.values)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    return -auc  # Negative because Optuna minimizes

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, timeout=3600)

# Display best parameters
print("Best parameters found:", study.best_params)

# Train the model with the best parameters
best_params = study.best_params
final_params = {
    "n_d": best_params["n_d"],
    "n_a": best_params["n_a"],
    "n_steps": best_params["n_steps"],
    "gamma": best_params["gamma"],
    "lambda_sparse": best_params["lambda_sparse"],
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=best_params["lr"]),
    "mask_type": best_params["mask_type"],
    "n_shared": best_params["n_shared"],
    "n_independent": best_params["n_independent"],
    "scheduler_params": dict(
        mode="min",
        patience=best_params["patienceScheduler"],
        min_lr=1e-5,
        factor=0.5,
    ),
    "scheduler_fn": torch.optim.lr_scheduler.ReduceLROnPlateau,
    "verbose": 1
}

clf = TabNetClassifier(**final_params)
clf.fit(
    X_train=X_train.values,
    y_train=y_train.values,
    eval_set=[(X_test.values, y_test.values)],
    eval_name=["valid"],
    patience=best_params["patience"],
    max_epochs=best_params["epochs"],
    batch_size=128,
    virtual_batch_size=64,
)

# Predictions
y_pred = clf.predict(X_test.values)
y_pred_proba = clf.predict_proba(X_test.values)[:, 1]

# Model evaluation
evaluate_model(y_test, y_pred, y_pred_proba, "TabNet with Optuna")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': clf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('TabNet - Feature Importance')
plt.tight_layout()
plt.show()

# Calibration curve
plt.figure(figsize=(8, 6))
prob_true, prob_pred = calibration_curve(y_test, y_pred_proba, n_bins=10)
plt.plot(prob_pred, prob_true, marker='o', label='TabNet')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly calibrated')
plt.xlabel('Mean predicted probability')
plt.ylabel('True probability')
plt.title('Calibration Curve - TabNet')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Save results
results_tabnet = pd.DataFrame({
    'True_Labels': y_test,
    'Predicted_Probabilities': y_pred_proba
})
results_tabnet.to_csv('tabnet_results.csv', index=False)


In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# StandardScaler'ı başlatın
scaler = StandardScaler()

# Eğitim verisini ölçeklendirin ve scaler'ı eğitin
X_train_scaled = scaler.fit_transform(X_train)

# Test verisini ölçeklendirin (Scaler'ı yeniden eğitmeyin)
X_test_scaled = scaler.transform(X_test)

# Hedef değişkenin sınıf sayısını kontrol etme
n_classes = len(np.unique(y_train))
print(f"Hedef değişkenin sınıf sayısı: {n_classes}")

# Model tahmin fonksiyonunu tanımlayın
def model_predict_shap(X):
    return clf.predict_proba(X)

# SHAP için arka plan verisi seçin (genellikle eğitim verisinin bir alt kümesi)
background = shap.sample(X_train_scaled, 100, random_state=42)

# SHAP KernelExplainer oluşturun
explainer = shap.KernelExplainer(model_predict_shap, background)

# SHAP değerlerini hesaplayın (bu işlem zaman alabilir)
print("SHAP değerleri hesaplanıyor, lütfen bekleyin...")
shap_values = explainer.shap_values(X_test_scaled, nsamples=100)

# SHAP Summary Plot (Bar)
print("SHAP Summary Plot oluşturuluyor...")
if n_classes == 2:
    # İkili sınıflandırma için
    if isinstance(shap_values, list):
        # SHAP değerleri bir liste ise, ikinci sınıf için kullan
        shap.summary_plot(shap_values[1], X_test_scaled, feature_names=X_train.columns, plot_type="bar")
    else:
        # SHAP değerleri tek bir dizi ise, üçüncü boyutta sınıfı seç
        shap.summary_plot(shap_values[:, :, 1], X_test_scaled, feature_names=X_train.columns, plot_type="bar")
else:
    # Çok sınıflı sınıflandırma için
    for class_idx in range(n_classes):
        print(f"\nSHAP Summary Plot oluşturuluyor - Sınıf {class_idx}:")
        if isinstance(shap_values, list):
            shap.summary_plot(shap_values[class_idx], X_test_scaled, feature_names=X_train.columns, plot_type="bar")
        else:
            shap.summary_plot(shap_values[:, :, class_idx], X_test_scaled, feature_names=X_train.columns, plot_type="bar")
        plt.title(f"SHAP Summary Plot - Class {class_idx}")
        plt.show()

# SHAP Summary Plot (Scatter)
print("SHAP Scatter Summary Plot oluşturuluyor...")
if n_classes == 2:
    if isinstance(shap_values, list):
        shap.summary_plot(shap_values[1], X_test_scaled, feature_names=X_train.columns)
    else:
        shap.summary_plot(shap_values[:, :, 1], X_test_scaled, feature_names=X_train.columns)
else:
    for class_idx in range(n_classes):
        shap.summary_plot(shap_values[class_idx], X_test_scaled, feature_names=X_train.columns)
        plt.title(f"SHAP Scatter Summary Plot - Class {class_idx}")
        plt.show()

# SHAP Dependence Plot
if n_classes == 2:
    top_feature = feature_importance.head(1)['feature'].values[0]
    print(f"{top_feature} için SHAP Dependence Plot oluşturuluyor...")
    if isinstance(shap_values, list):
        shap.dependence_plot(top_feature, shap_values[1], X_test_scaled, feature_names=X_train.columns)
    else:
        shap.dependence_plot(top_feature, shap_values[:, :, 1], X_test_scaled, feature_names=X_train.columns)
    plt.title(f"SHAP Dependence Plot - {top_feature}")
    plt.show()
else:
    for class_idx in range(n_classes):
        top_feature = feature_importance.head(1)['feature'].values[0]
        print(f"{top_feature} için SHAP Dependence Plot oluşturuluyor - Sınıf {class_idx}:")
        if isinstance(shap_values, list):
            shap.dependence_plot(top_feature, shap_values[class_idx], X_test_scaled, feature_names=X_train.columns)
        else:
            shap.dependence_plot(top_feature, shap_values[:, :, class_idx], X_test_scaled, feature_names=X_train.columns)
        plt.title(f"SHAP Dependence Plot - Class {class_idx}")
        plt.show()

# SHAP Force Plot (İlk birkaç örnek için)
print("SHAP Force Plot oluşturuluyor...")
shap.initjs()
for i in range(min(3, X_test_scaled.shape[0])):  # İlk 3 örnek için
    if n_classes == 2:
        if isinstance(shap_values, list):
            shap.force_plot(explainer.expected_value[1], shap_values[1][i], X_test_scaled[i], matplotlib=True)
        else:
            shap.force_plot(explainer.expected_value[1], shap_values[i, :, 1], X_test_scaled[i], matplotlib=True)
    else:
        for class_idx in range(n_classes):
            if isinstance(shap_values, list):
                shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx][i], X_test_scaled[i], matplotlib=True)
            else:
                shap.force_plot(explainer.expected_value[class_idx], shap_values[i, :, class_idx], X_test_scaled[i], matplotlib=True)
    plt.show()


In [ ]:
import joblib

# Scaler'ı kaydedin
joblib.dump(scaler, 'scaler.joblib')


['scaler.joblib']

In [ ]:
# Tüm modeli kaydedin
torch.save(clf, 'tabnet_model.pth')


# KAN

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import matthews_corrcoef, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
#import shap

# Veri yükleme
df = pd.read_excel('/content/df.xlsx')

# Train ve test verilerini ayırma
train_data = df[df['region'] == 1].copy()
test_data = df[df['region'] == 2].copy()

# Kategorik ve numerik değişkenleri ayırma
categorical_features = ['Multifocal_PVC', 'Nonsustained_VT', 'gender', 'HTN', 'DM', 'Fullcompansasion']
numeric_features = ['pvc_percent', 'PVCQRS', 'EF', 'Age', 'PVC_Prematurity_index', 'QRS_ratio',
                   'mean_HR', 'symptom_duration', 'QTc_sinus', 'PVCCI_dispersion', 'CI_variability',
                   'PVC_Peak_QRS_duration', 'PVCCI', 'PVC_Compansatory_interval']

# Label Encoding için kategorik değişkenleri dönüştürme
le = LabelEncoder()
for col in categorical_features:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.transform(test_data[col])

# Standartlaştırma
scaler = StandardScaler()
train_data[numeric_features] = scaler.fit_transform(train_data[numeric_features])
test_data[numeric_features] = scaler.transform(test_data[numeric_features])

# X ve y ayırma
X_train = train_data[numeric_features + categorical_features]
y_train = train_data['Group']
X_test = test_data[numeric_features + categorical_features]
y_test = test_data['Group']

def evaluate_model(y_true, y_pred, y_pred_proba, model_name):
    """
    Model performans metriklerini hesaplar ve yazdırır
    """
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred_proba)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    print(f"\n{model_name} Performans Metrikleri:")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"AUC: {auc:.3f}")
    print(f"Sensitivity: {sensitivity:.3f}")
    print(f"Specificity: {specificity:.3f}")
    print(f"PPV: {ppv:.3f}")
    print(f"NPV: {npv:.3f}")
    print(f"F1 Score: {f1:.3f}")
    print(f"MCC: {mcc:.3f}")

    return auc

def plot_roc_curves(models_dict):
    """
    Tüm modeller için ROC eğrilerini çizer
    """
    plt.figure(figsize=(10, 8))
    for model_name, (model, y_pred_proba) in models_dict.items():
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        auc = roc_auc_score(y_test, y_pred_proba)
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.3f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for All Models')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'feature_names': X_train.columns.tolist(),
    'X_train_scaled': X_train_scaled,
    'X_test_scaled': X_test_scaled,
    'y_test': y_test
}, 'kan_model_full.pth')

In [ ]:
# Eğitim sonrası model kaydetme
X_train_np = X_train_scaled
X_test_np = X_test_scaled

if isinstance(X_train_np, np.ndarray):
    X_train_np = X_train_np.astype(np.float32)
if isinstance(X_test_np, np.ndarray):
    X_test_np = X_test_np.astype(np.float32)

torch.save({
    'model_state_dict': model.state_dict(),
    'feature_names': X_train.columns.tolist(),
    'X_train': X_train_np,
    'X_test': X_test_np
}, 'kan_model.pth')

In [ ]:
def analyze_shap(model_path):
    import torch
    import shap
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt

    # Define or import KANWrapper if not already defined
    class KANWrapper:
        def __init__(self, model, device):
            self.model = model
            self.device = device

        def __call__(self, data):
            self.model.eval()
            with torch.no_grad():
                data = torch.FloatTensor(data).to(self.device)
                outputs = self.model(data)
                probabilities = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            return probabilities

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the checkpoint with map_location to ensure compatibility
    checkpoint = torch.load(model_path, map_location=device)

    # Initialize the model architecture
    input_dim = len(checkpoint['feature_names'])
    hidden_dims = [64, 32]
    inner_dim = 32
    model = KolmogorovArnoldNetwork(input_dim, hidden_dims, inner_dim).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Prepare background and explain data
    background_data = checkpoint['X_train'][:100]
    explain_data = checkpoint['X_test'][:100]

    # Convert feature_names to a NumPy array
    feature_names = np.array(checkpoint['feature_names'])

    # Initialize SHAP KernelExplainer
    wrapper = KANWrapper(model, device)
    explainer = shap.KernelExplainer(wrapper, background_data)

    # Calculate SHAP values
    shap_values = explainer.shap_values(explain_data)

    # Handle multi-class or binary classification
    if isinstance(shap_values, list):
        shap_values_plot = shap_values[1]
    else:
        shap_values_plot = shap_values

    # Visualization: SHAP Summary Plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(
        shap_values_plot,
        explain_data,
        feature_names=feature_names,
        show=False
    )

    # Feature Importance DataFrame
    feature_importance = np.abs(shap_values_plot).mean(axis=0)
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)

    # Save the summary plot
    plt.tight_layout()
    plt.savefig('shap_summary.png')
    plt.close()

    return shap_values, importance_df

In [ ]:
shap_values, importance_df = analyze_shap('kan_model.pth')

# Display feature importance
print(importance_df)

# Kolmogorov Arnold Network

In [ ]:
import optuna
import torch.nn as nn
# GPU kullanılabilirse GPU'yu, değilse CPU'yu kullan
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class KolmogorovArnoldNetworkWithDropout(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32], inner_dim=32, dropout_rate=0.2):
        super().__init__()
        self.input_dim = input_dim

        # KAN layers with dropout
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(KolmogorovArnoldLayer(prev_dim, inner_dim, hidden_dim))
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim

        self.kan_layers = nn.ModuleList(layers)
        self.output_layer = nn.Linear(hidden_dims[-1], 2)

    def forward(self, x):
        for layer in self.kan_layers:
            x = layer(x)
        return self.output_layer(x)

def objective(trial):
    # Hiperparametreleri tanımla
    hidden_dims = [
        trial.suggest_int('hidden_dim1', 32, 256),
        trial.suggest_int('hidden_dim2', 16, 128)
    ]
    inner_dim = trial.suggest_int('inner_dim', 16, 128)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])

    # Dataset ve DataLoader oluştur
    train_dataset = KANDataset(X_train_scaled, y_train.values)
    valid_dataset = KANDataset(X_test_scaled, y_test.values)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

    # Model oluştur
    model = KolmogorovArnoldNetworkWithDropout(
        input_dim=X_train.shape[1],
        hidden_dims=hidden_dims,
        inner_dim=inner_dim,
        dropout_rate=dropout_rate
    ).to(device)

    # Optimizer ve loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Early stopping parametreleri
    patience = 5
    best_auc = 0
    patience_counter = 0

    # Eğitim döngüsü
    for epoch in range(30):  # Maximum 30 epoch
        # Train
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        all_probs = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                probs = torch.softmax(outputs, dim=1)
                all_probs.extend(probs[:, 1].cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        current_auc = roc_auc_score(all_labels, all_probs)

        # Early stopping kontrolü
        if current_auc > best_auc:
            best_auc = current_auc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    return best_auc

# Optuna çalıştırma
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# En iyi hiperparametreler
print("En iyi hiperparametreler:")
print(study.best_params)
print(f"\nEn iyi AUC skoru: {study.best_value:.4f}")

# Hiperparametre önem analizi
importance = optuna.importance.get_param_importances(study)
print("\nHiperparametre önem sıralaması:")
for param, score in importance.items():
    print(f"{param}: {score:.4f}")

# En iyi modeli eğit
best_params = study.best_params
final_model = KolmogorovArnoldNetworkWithDropout(
    input_dim=X_train.shape[1],
    hidden_dims=[best_params['hidden_dim1'], best_params['hidden_dim2']],
    inner_dim=best_params['inner_dim'],
    dropout_rate=best_params['dropout_rate']
).to(device)

# Final modeli en iyi hiperparametrelerle eğit
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'])
optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])

# Model eğitimi ve değerlendirmesi
train_model(final_model, train_loader, criterion, optimizer, device)
_, _, y_pred_proba, y_true = evaluate_model(final_model, test_loader, criterion, device)
y_pred = (y_pred_proba > 0.5).astype(int)

# Final metrikleri hesapla
print("\nOptimize edilmiş model performansı:")
calculate_metrics(y_true, y_pred, y_pred_proba, "Optimized KAN")

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'feature_names': X_train.columns.tolist(),
    'X_train_scaled': X_train_scaled,
    'X_test_scaled': X_test_scaled,
    'y_test': y_test
}, 'kan_model_2.pth')

In [ ]:
def analyze_shap(model_path):
    import torch
    import shap
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt

    # Define KANWrapper
    class KANWrapper:
        def __init__(self, model, device):
            self.model = model
            self.device = device

        def __call__(self, data):
            self.model.eval()
            with torch.no_grad():
                data = torch.FloatTensor(data).to(self.device)
                outputs = self.model(data)
                probabilities = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            return probabilities

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the checkpoint
    checkpoint = torch.load(model_path, map_location=device)

    # Initialize the model with optimized hyperparameters
    input_dim = len(checkpoint['feature_names'])
    hidden_dims = [94, 55]  # Optimize edilmiş değerler
    inner_dim = 37  # Optimize edilmiş değer
    dropout_rate = 0.4669  # Optimize edilmiş değer

    # Initialize model with optimized architecture
    model = KolmogorovArnoldNetworkWithDropout(
        input_dim=input_dim,
        hidden_dims=hidden_dims,
        inner_dim=inner_dim,
        dropout_rate=dropout_rate
    ).to(device)

    # Load state dict
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Prepare background and explain data
    background_data = checkpoint['X_train_scaled'][:100]  # Using scaled data
    explain_data = checkpoint['X_test_scaled'][:100]  # Using scaled data

    # Convert feature_names to a NumPy array
    feature_names = np.array(checkpoint['feature_names'])

    # Initialize SHAP KernelExplainer
    wrapper = KANWrapper(model, device)
    explainer = shap.KernelExplainer(wrapper, background_data)

    # Calculate SHAP values
    print("Calculating SHAP values (this may take a while)...")
    shap_values = explainer.shap_values(explain_data)

    # Handle multi-class or binary classification
    if isinstance(shap_values, list):
        shap_values_plot = shap_values[1]
    else:
        shap_values_plot = shap_values

    # Visualization: SHAP Summary Plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(
        shap_values_plot,
        explain_data,
        feature_names=feature_names,
        show=False
    )

    # Feature Importance DataFrame
    feature_importance = np.abs(shap_values_plot).mean(axis=0)
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)

    # Save the summary plot
    plt.tight_layout()
    plt.savefig('shap_summary.png')
    plt.close()

    print("SHAP analysis completed. Results saved as 'shap_summary.png'")
    return shap_values, importance_df
def plot_shap_importance(importance_df):
    # Veriyi önemlilik sırasına göre sırala
    df_sorted = importance_df.sort_values('importance', ascending=True)

    # Görselleştirme
    plt.figure(figsize=(12, 10))

    # Barplot oluştur
    bars = plt.barh(range(len(df_sorted)), df_sorted['importance'])

    # Y ekseni etiketlerini ayarla
    plt.yticks(range(len(df_sorted)), df_sorted['feature'])

    # Başlık ve eksen etiketleri
    plt.title('Feature Importance Based on SHAP Values', pad=20)
    plt.xlabel('mean(|SHAP value|) (average impact on model output magnitude)')

    # Çubukları mavi yap
    for bar in bars:
        bar.set_color('#1f77b4')  # Matplotlib'in varsayılan mavi rengi

    # Izgarayı ekle ve stilini ayarla
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    # Layout'u düzenle
    plt.tight_layout()

    # Grafiği kaydet
    plt.savefig('shap_importance_barplot.png', dpi=300, bbox_inches='tight')
    plt.close()

    print("Barplot saved as 'shap_importance_barplot.png'")

# Görselleştirmeyi çalıştır
plot_shap_importance(importance_df)
# Run SHAP analysis
shap_values, importance_df = analyze_shap('kan_model_2.pth')

# Display feature importance
print("\nFeature Importance Rankings:")
print(importance_df)